# Introducción

Crea las series diarias de temperatura máxima, mínima y media de las estaciones de la AEMET en toda España en el periodo 1950-2015.
Los datos provienen de una serie de archivos en las que se dividen por zonas del país y periodos de varios años

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#from mpl_toolkits.basemap import Basemap, cm
import numpy as np
import glob
import sys
import tqdm
from dateutil import parser
import datetime
from itertools import chain

In [9]:
# Variable
var = 'Pd'

# Ruta de los datos y carpetas
ruta_series = 'F:/Series/AEMET/2016_pet080_UNICAN/data/'
folders = ['Ind_0001_3015', 'Ind_3018E_6250', 'Ind_6251_9812E', 'Ind_9813_C939U']
ruta_exportar = 'F:/Series/AEMET/2016_pet080_UNICAN/data/Precipitacion/'

# Define el periodo de estudio
time = pd.date_range(start='1950-01-01', end='2015-12-31', freq='D')

### Extraer las estaciones

Se extraen las estaciones con alguna medición y sus características (código, nombre, coordenadas, altitud, provincia).

__Lista de archivos__

In [10]:
files = []
for folder in folders:
    # Genera la serie de rutas de los archivos
    f = glob.glob(ruta_series + folder + '/Datos-' + var + '*')
    f = np.sort(f)
    files.append(f)
    del f
# Convertir la lista a una dimensión
files = list(chain.from_iterable(files))

print('Nº de archivos:', len(files))

Nº de archivos: 61


In [11]:
# Extraer características de las estaciones
estaciones = pd.DataFrame(columns=['INDICATIVO', 'NOMBRE', 'ALTITUD', 'C_X', 'C_Y', 'NOM_PROV', 'LONGITUD', 'LATITUD'])
estaciones.set_index('INDICATIVO', drop=True, inplace=True)

# Buscar estaciones y sus características en todos los archivos
for file in tqdm.tqdm(files):
    aux = pd.read_csv(file, sep=';', encoding='latin-1', usecols=[0, 4, 5, 6, 7, 8, 9 , 10], index_col=0,
                      dtype={'INDICATIVO': str, 'NOM_PROV': str, 'LONGITUD': str})
    for stn in aux.index.unique():
        if stn not in estaciones.index:
            estaciones = pd.concat((estaciones, pd.DataFrame(aux.loc[stn,:].iloc[0]).T), axis=0, join='outer')
            
# Exportar las características de las estaciones
estaciones.index.name = 'INDICATIVO'
estaciones.to_csv(ruta_exportar + 'Estaciones.csv')

print('Nº de estaciones:', estaciones.shape[0])

100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [00:43<00:00,  1.41it/s]


Nº de estaciones: 3381


### Generar las series diarias

Se importan los datos del conjunto de archivos antes definidos, se concatenan las distintas series temporales de cada estación y se unen todas las estaciones en un mismo "data frame" para cada variable.

In [13]:
# Crea los "data frame" con los nombre de la carpeta en el que se guardarán todas las series, siendo las files la
# fecha y las columnas las estaciones
Pd = pd.DataFrame(index=time, columns=estaciones.index)
Pd.index.name = 'FECHA'

In [ ]:
# Completa el data frame
for file in tqdm.tqdm(files):
    aux = pd.read_csv(file, sep=';', encoding='latin-1', dtype={'INDICATIVO': str, 'NOM_PROV': str, 'LONGITUD': str})
    ids = []
    ids_t = []
    for xx, x in enumerate(aux.INDICATIVO):
        if isinstance(x, float): # si 'x' es real, convierte en entero
            x = int(x); 
        if isinstance(x, int): # si 'x' es un entero, convierte en caracteres
            x = str(x);  
        ids_t.append(x)
        if x not in ids:
            ids.append(x) #no los guardo si ya están repetidos
    aux.INDICATIVO = ids_t    

    for x in ids: # para cada estación
        # Genera un "data frame" con sólo las filas de la estación en cuestión
        aux_cut = aux.copy()
        aux_cut = aux_cut.loc[np.where(aux_cut.INDICATIVO == x)[0]]
        # Crea una serie de fechas en las que la estación tiene datos
        time_aux = [datetime.datetime(aux_cut.AÑO.values[t], aux_cut.MES.values[t], aux_cut.DIA.values[t]) 
                    for t in range(len(aux_cut.AÑO))]
        # Crea el "data frame" con tantas filas como fechas hay dato para la estación
        dataframe_aux = pd.DataFrame(index=time_aux, columns=['P77'])
        # Completar la serie con los datos
        dataframe_aux.P77 = aux_cut.P77.values
        # Asignar los datos a la matriz de la magnitud correspondiente
        Pd[x].loc[dataframe_aux.index] = dataframe_aux.P77.values

  3%|██▋                                                                                | 2/61 [01:48<53:12, 54.11s/it]

In [ ]:
# Exporta las series completas de temperatura máxima, mínima y media dentro de la carpeta
Pd.to_csv(ruta_exportar + 'Pd_1950.csv')

In [ ]:
print('Nº de estaciones:', len(Pd.columns.unique()))
print('Nº de días:', Pd.shape[0])